### credits to "https://realpython.com/inner-functions-what-are-they-good-for/"

# Encapsulation


In [3]:
def outer(num1):
    def inner_increment(num1):  # Hidden from outer code
        return num1 + 1
    num2 = inner_increment(num1)
    print(num1, num2)

#inner_increment(10)
outer(10)

10 11


### Note: Keep in mind that this is just an example. Although this code does achieve the desired result, it’s probably better to make inner_increment() a top-level “private” function using a leading underscore: "	&#95;inner&#95;increment()".

In [4]:
def factorial(number):

    # Error handling
    if not isinstance(number, int):
        raise TypeError("Sorry. 'number' must be an integer.")
    if not number >= 0:
        raise ValueError("Sorry. 'number' must be zero or positive.")

    def inner_factorial(number):
        if number <= 1:
            return 1
        return number*inner_factorial(number-1)
    return inner_factorial(number)

In [5]:
print(factorial(4))

24


In [6]:
## Perhaps you have a giant function that performs the same chunk of code in numerous places.  
## For example, you might write a function that processes a file, 
## and you want to accept either an open file object or a file name:

def process(file_name):
    def do_stuff(file_process):
        for line in file_process:
            print(line)
    if isinstance(file_name, str):
        with open(file_name, 'r') as f:
            do_stuff(f)
    else:
        do_stuff(file_name)
        
## Note: Again, it is common to just make do_stuff() a private top-level function,
## but if you want to hide it away as an internal function, you can.

In [7]:
## Let’s say you want to know the number of WiFi hotspots in New York City. Yes, the city has the raw data to tell us. 
## Visit the site and download the CSV:

def process(file_name):

    def do_stuff(file_process):
        wifi_locations = {}

        for line in file_process:
            values = line.split(',')
            # Build the dict and increment values.
            wifi_locations[values[1]] = wifi_locations.get(values[1], 0) + 1

        max_key = 0
        for name, key in wifi_locations.items():
            all_locations = sum(wifi_locations.values())
            if key > max_key:
                max_key = key
                business = name

        print(f'There are {all_locations} WiFi hotspots in NYC, '
              f'and {business} has the most with {max_key}.')

    if isinstance(file_name, str):
        with open(file_name, 'r') as f:
            do_stuff(f)
    else:
        do_stuff(file_name)

Closures and Factory Functions
Now we come to the most important reason to use inner functions. 
All of the inner function examples we’ve seen so far have been ordinary functions that merely happened to be nested inside another function. 
In other words, we could have defined these functions in another way (as discussed). 
There is no specific reason for why they need to be nested.

But when it comes to closures, that is not the case: you must use nested functions.

What’s a Closure?
A closure simply causes the inner function to remember the state of its environment when called. 
Beginners often think that a closure is the inner function, but it’s really caused by the inner function. 
The closure “closes” the local variable on the stack, and this stays around after the stack creation has finished executing.

In [8]:
def generate_power(number):
    """
    Examples of use:

    >>> raise_two = generate_power(2)
    >>> raise_three = generate_power(3)
    >>> print(raise_two(7))
    128
    >>> print(raise_three(5))
    243
    """

    # Define the inner function ...
    def nth_power(power):
        return number ** power
    # ... that is returned by the factory function.

    return nth_power


In [9]:
raise_two = generate_power(2) # this sets the number we want to get its nth power
print(raise_two(3)) # this line computes the 3rd power of number = 2 

8


What’s Happening in the Example
Let’s take a look at what is going on in that example:

generate_power() is a factory function, which simply means that it creates a new function each time it is called and then returns the newly created function. Thus, raise_two() and raise_three() are the newly created functions.
What does this new, inner function do? It takes a single argument, power, and returns number**power.
Where does the inner function get the value of number from? This is where the closure comes into play: nth_power() gets the value of power from the outer function, the factory function. Let’s step through this process:

Call the outer function: generate_power(2).
Build nth_power(), which takes a single argument power.
Take a snapshot of the state of nth_power(), which includes power=2.
Pass that snapshot into generate_power().
Return nth_power().
To put it another way, the closure “initializes” the number bar in nth_power() and then returns it. Now, whenever you call that newly returned function, it will always see its own private snapshot that includes power=2.

In [10]:
## This is a simplified function to check if a certain user has the correct permissions to access a certain page. 
## You could easily modify this to grab the user in session to check if they have the correct credentials to access a certain route. 
## Instead of checking if the user is just equal to 'Admin', 
## you could query the database to check the permission and then return 
## the correct view depending on whether the credentials are correct or not.


def has_permission(page):
    def inner(username):
        if username == 'Admin':
            return "'{0}' does have access to {1}.".format(username, page)
        else:
            return "'{0}' does NOT have access to {1}.".format(username, page)
    return inner


current_user = has_permission('Admin Area')
print(current_user('Admin'))

random_user = has_permission('Admin Area')
print(random_user('Not Admin'))

'Admin' does have access to Admin Area.
'Not Admin' does NOT have access to Admin Area.


In [11]:
## The use of closures and factory functions is the most common and powerful use for inner functions. 
## In most cases, when you see a decorated function, the decorator is a factory 
## function that takes a function as argument and returns a new function that includes the old function 
## inside the closure.

## To put it another way, a decorator is just syntactic sugar for 
## implementing the process outlined in the generate_power() example.


def generate_power(exponent):
    def decorator(f):
        def inner(*args):
            result = f(*args)
            return exponent**result
        return inner
    return decorator

In [12]:
@generate_power(2)
def raise_two(n):
    return n

print(raise_two(7))


@generate_power(3)
def raise_three(n):
    return n

print(raise_three(2))

128
9
